# Baseball Games- SVM Classification

we will focus on sports analytics for this project. This data set is made available by http://www.baseball-reference.com. It contains data about professional baseball (MLB) games played in the 2016 season. There are 2,427 games in the data set. Each row represents a single game. The goal is to predict the attendance at a home team’s game. This is an important task because most franchises want to predict the number of attendees for a variety of reasons including profits.

## Description of Variables

The description of variables are provided in "Baseball - Data Dictionary.docx"

## attendance_binary Goal

Use the **baseball.csv** data set and build a model to predict **attendance_binary**.

# Read and Prepare the Data:

In [5]:
# Common imports
import numpy as np
import pandas as pd

np.random.seed(42)


In [6]:
#We will predict the "price_gte_150" value in the data set:

baseball = pd.read_csv("baseball.csv")
baseball.head()

,attendance_binary,previous_attendance,previous_away_team_errors,previous_away_team_hits,previous_away_team_runs,game_type,previous_game_type,previous_home_team_errors,previous_home_team_hits,previous_home_team_runs,game_day,previous_game_day,temperature,wind_speed,sky,previous_game_duration,previous_homewin
0,0,43683,2,6,2,Night Game,Day Game,0,6,6,Wednesday,Monday,55,24,Overcast,2.933333,1
1,0,45785,0,7,2,Night Game,Day Game,0,10,3,Wednesday,Monday,48,7,Unknown,2.800000,1
2,0,48282,0,8,4,Night Game,Day Game,2,4,3,Wednesday,Monday,65,10,Cloudy,3.383333,0
3,0,21830,0,9,6,Day Game,Night Game,0,15,11,Wednesday,Tuesday,77,0,In Dome,3.233333,1
4,0,49289,2,4,2,Night Game,Day Game,1,1,3,Tuesday,Monday,81,12,Cloudy,2.633333,1


In [7]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(baseball, test_size=0.3)

In [8]:
train_set.isna().sum()

attendance_binary            0
previous_attendance          0
previous_away_team_errors    0
previous_away_team_hits      0
previous_away_team_runs      0
game_type                    0
previous_game_type           0
previous_home_team_errors    0
previous_home_team_hits      0
previous_home_team_runs      0
game_day                     0
previous_game_day            0
temperature                  0
wind_speed                   0
sky                          0
previous_game_duration       0
previous_homewin             0
dtype: int64

In [9]:
test_set.isna().sum()

attendance_binary            0
previous_attendance          0
previous_away_team_errors    0
previous_away_team_hits      0
previous_away_team_runs      0
game_type                    0
previous_game_type           0
previous_home_team_errors    0
previous_home_team_hits      0
previous_home_team_runs      0
game_day                     0
previous_game_day            0
temperature                  0
wind_speed                   0
sky                          0
previous_game_duration       0
previous_homewin             0
dtype: int64

#### Data Prep

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

#### Separate the target variable (we don't want to transform it)

In [11]:
train_y = train_set[['attendance_binary']]
test_y = test_set[['attendance_binary']]

train_inputs = train_set.drop(['attendance_binary'], axis=1)
test_inputs = test_set.drop(['attendance_binary'], axis=1)

#### Identify Numerical and Categorical Column 

In [12]:
train_inputs.dtypes

previous_attendance            int64
previous_away_team_errors      int64
previous_away_team_hits        int64
previous_away_team_runs        int64
game_type                     object
previous_game_type            object
previous_home_team_errors      int64
previous_home_team_hits        int64
previous_home_team_runs        int64
game_day                      object
previous_game_day             object
temperature                    int64
wind_speed                     int64
sky                           object
previous_game_duration       float64
previous_homewin               int64
dtype: object

In [13]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [14]:
# Identify the binary columns so we can pass them through without transforming
binary_columns = ['previous_homewin']

In [15]:
for col in binary_columns:
    numeric_columns.remove(col)

In [16]:
binary_columns

['previous_homewin']

In [17]:
numeric_columns

['previous_attendance',
 'previous_away_team_errors',
 'previous_away_team_hits',
 'previous_away_team_runs',
 'previous_home_team_errors',
 'previous_home_team_hits',
 'previous_home_team_runs',
 'temperature',
 'wind_speed',
 'previous_game_duration']

In [18]:
categorical_columns

['game_type', 'previous_game_type', 'game_day', 'previous_game_day', 'sky']

### Pipeline

In [19]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [20]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [21]:
binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))])

In [22]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns),
        ('binary', binary_transformer, binary_columns)],
        remainder='passthrough')

### Transform: fit_transform() for TRAIN

In [23]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)

train_x

array([[-1.12371621,  0.57666325, -0.20719118, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.05787587, -0.72716391, -0.78017264, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38394295,  0.57666325, -1.35315411, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-0.52534761,  0.57666325, -0.78017264, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.94392488, -0.72716391,  0.65228102, ...,  0.        ,
         0.        ,  0.        ],
       [-0.98407336,  1.88049041, -0.49368191, ...,  1.        ,
         0.        ,  1.        ]])

In [24]:
train_x.shape

(1698, 37)

### Tranform: transform() for TEST 

In [25]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x

array([[ 0.0814842 ,  1.88049041, -1.06666338, ...,  1.        ,
         0.        ,  1.        ],
       [ 1.17001331, -0.72716391, -1.63964484, ...,  0.        ,
         0.        ,  0.        ],
       [-0.29905768, -0.72716391, -0.20719118, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-0.49582715, -0.72716391,  1.22526249, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.26445059, -0.72716391, -0.20719118, ...,  0.        ,
         0.        ,  1.        ],
       [-1.71775245, -0.72716391, -1.63964484, ...,  0.        ,
         0.        ,  0.        ]])

In [26]:
test_x.shape

(729, 37)

# Calculate the Baseline

In [27]:
# Find majority class
train_y.value_counts()

attendance_binary
1                    873
0                    825
dtype: int64

In [28]:
# Find percentage
train_y.value_counts()/len(train_y)

attendance_binary
1                    0.514134
0                    0.485866
dtype: float64

# SVM Model 1:

## SVC(kernel='linear')

In [29]:
from sklearn.svm import SVC
 
lin_svm2 = SVC(kernel="linear")

lin_svm2.fit(train_x, train_y)

C:\Users\anush\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


SVC(kernel='linear')

### Accuracy

In [30]:
from sklearn.metrics import accuracy_score

In [31]:
#Predict the train values
train_y_pred = lin_svm2.predict(train_x)

#Train accuracy
accuracy_score(train_y, train_y_pred)

0.8445229681978799

In [32]:
#Predict the test values
test_y_pred = lin_svm2.predict(test_x)

#Test accuracy
accuracy_score(test_y, test_y_pred)

0.8175582990397805

### Classification Matrix 

In [33]:
from sklearn.metrics import confusion_matrix

#We usually create the confusion matrix on test set
confusion_matrix(test_y, test_y_pred)

array([[278,  65],
       [ 68, 318]], dtype=int64)

### Classification Report

In [34]:
from sklearn.metrics import classification_report

#We usually create the classification report on test set
print(classification_report(test_y, test_y_pred))

              precision    recall  f1-score   support

           0       0.80      0.81      0.81       343
           1       0.83      0.82      0.83       386

    accuracy                           0.82       729
   macro avg       0.82      0.82      0.82       729
weighted avg       0.82      0.82      0.82       729



# SVM Model 2:

## SVC(kernel='poly') 

In [35]:
from sklearn.svm import SVC

# You need to enter a value for gamma. Remember, gamma controls the shape of the bell curve for rbf
# You can also set it is as gamma='scale'. This will be the default option in future releases

pol_svm2 = SVC(kernel="poly", degree=2, coef0=1, C=1, gamma='scale')

pol_svm2.fit(train_x, train_y)

C:\Users\anush\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


SVC(C=1, coef0=1, degree=2, kernel='poly')

### Accuracy 

In [36]:
#Predict the train values
train_y_pred = pol_svm2.predict(train_x)

#Train accuracy
accuracy_score(train_y, train_y)

1.0

In [37]:
#Predict the test values
test_y_pred = pol_svm2.predict(test_x)

#Test accuracy
accuracy_score(test_y, test_y_pred)

0.8244170096021948

In [38]:
##overfitting doing great with training set not with test dataset


### Classification Matrix

In [39]:
from sklearn.metrics import confusion_matrix

#We usually create the confusion matrix on test set
confusion_matrix(test_y, test_y_pred)

array([[283,  60],
       [ 68, 318]], dtype=int64)

### Classification Report 

In [40]:
from sklearn.metrics import classification_report

#We usually create the classification report on test set
print(classification_report(test_y, test_y_pred))

              precision    recall  f1-score   support

           0       0.81      0.83      0.82       343
           1       0.84      0.82      0.83       386

    accuracy                           0.82       729
   macro avg       0.82      0.82      0.82       729
weighted avg       0.82      0.82      0.82       729



# SVM Model 3 

## SVC(kernel='rbf')

In [41]:
rbf_svm = SVC(kernel="rbf", C=150, gamma='scale')

rbf_svm.fit(train_x, train_y)

C:\Users\anush\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


SVC(C=150)

### Accuracy 

In [42]:
#Predict the train values
train_y_pred = rbf_svm.predict(train_x)

#Train accuracy
accuracy_score(train_y, train_y_pred)

0.9994110718492344

In [43]:
#Predict the test values
test_y_pred = rbf_svm.predict(test_x)

#Test accuracy--Overfitting model
accuracy_score(test_y, test_y_pred)

0.7805212620027435

### Classification Matrix 

In [44]:
from sklearn.metrics import confusion_matrix

#We usually create the confusion matrix on test set
confusion_matrix(test_y, test_y_pred)

array([[271,  72],
       [ 88, 298]], dtype=int64)

### Classification Report 

In [45]:
from sklearn.metrics import classification_report

#We usually create the classification report on test set
print(classification_report(test_y, test_y_pred))

              precision    recall  f1-score   support

           0       0.75      0.79      0.77       343
           1       0.81      0.77      0.79       386

    accuracy                           0.78       729
   macro avg       0.78      0.78      0.78       729
weighted avg       0.78      0.78      0.78       729



# SVM Model 4

## Grid Search: Randomized

In [46]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
import random

param_distribs = {
        'C': randint(low=5, high=50),
        'gamma': uniform(0.1, 0.5),    
    }

rbf_svm = SVC(kernel="rbf", decision_function_shape='ovr')

grid_search = RandomizedSearchCV(rbf_svm, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='accuracy', random_state=42)

grid_search.fit(train_x, train_y)

C:\Users\anush\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\anush\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\anush\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\anush\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

C:\Users\anush\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\anush\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\anush\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\anush\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

RandomizedSearchCV(cv=5, estimator=SVC(),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000022C1820FF70>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000022C18226040>},
                   random_state=42, scoring='accuracy')

In [48]:
cvres = grid_search.cv_results_

for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

0.7732778066979005 {'C': 43, 'gamma': 0.49827149343011645}
0.7826999826479264 {'C': 19, 'gamma': 0.4659969709057026}
0.7897622765920527 {'C': 25, 'gamma': 0.17800932022121826}
0.77974492451848 {'C': 23, 'gamma': 0.14998745790900145}
0.7632726010758286 {'C': 15, 'gamma': 0.5330880728874676}
0.7897605413846955 {'C': 40, 'gamma': 0.1714334089609704}
0.7897501301405518 {'C': 7, 'gamma': 0.11029224714790123}
0.7844664237376368 {'C': 6, 'gamma': 0.4609993861334124}
0.7950633350685407 {'C': 34, 'gamma': 0.20616955533913808}
0.7950650702758979 {'C': 25, 'gamma': 0.4087407548138583}


### Accuracy 

In [49]:
final_model = grid_search.best_estimator_

test_predictions = final_model.predict(test_x)

#Test accuracy
accuracy_score(test_y, test_predictions)

0.8024691358024691

### Classification Matrix

In [50]:
from sklearn.metrics import confusion_matrix

#We usually create the confusion matrix on test set
confusion_matrix(test_y, test_y_pred)

array([[271,  72],
       [ 88, 298]], dtype=int64)

## Classification Report

In [51]:
from sklearn.metrics import classification_report

#We usually create the classification report on test set
print(classification_report(test_y, test_y_pred))

              precision    recall  f1-score   support

           0       0.75      0.79      0.77       343
           1       0.81      0.77      0.79       386

    accuracy                           0.78       729
   macro avg       0.78      0.78      0.78       729
weighted avg       0.78      0.78      0.78       729



# Discussion

1) Which model performs the best (and why)?<br>
2) Does the best model perform better than the baseline (and why)?<br>
3) Does the best model exhibit any overfitting; what did you do about it?

In [1]:
##